In [124]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337
date 20140526


In [134]:
from copy import copy


def proto1(feed, trip_stats, dates):
    frames = []
    for date in dates:
        f = feed.compute_feed_stats(ts, date)
        f['date'] = date
        frames.append(f)
    
    return pd.concat(frames)

def proto2(feed, trip_stats, dates):
    """
    Given trip stats of the form output by :func:`compute_trip_stats` and a date, return a DataFrame including the following feed stats for the date.

    - num_trips: number of trips active on the given date
    - num_routes: number of routes active on the given date
    - num_stops: number of stops active on the given date
    - peak_num_trips: maximum number of simultaneous trips in service
    - peak_start_time: start time of first longest period during which
      the peak number of trips occurs
    - peak_end_time: end time of first longest period during which
      the peak number of trips occurs
    - service_distance: sum of the service distances for the active routes
    - service_duration: sum of the service durations for the active routes
    - service_speed: service_distance/service_duration

    If there are no stats for the given date, return an empty DataFrame with the specified columns.

    Assume the following feed attributes are not ``None``:

    - Those used in :func:`get_trips`
    - Those used in :func:`get_routes`
    - Those used in :func:`get_stops`

    """
    hp = gt
    cols = [
      'date',
      'num_trips',
      'num_routes',
      'num_stops',
      'peak_num_trips',
      'peak_start_time',
      'peak_end_time',
      'service_distance',
      'service_duration',
      'service_speed',
    ]
    stats = trip_stats.copy()
    activity = feed.compute_trip_activity(dates)
    stop_times = feed.stop_times.copy()
    
    # Convert timestrings to seconds for quicker calculations
    stats[['start_time', 'end_time']] =\
      stats[['start_time', 'end_time']].applymap(hp.timestr_to_seconds)
    
    # Compile stats for each date, but memoize stats by trip ID sequence 
    # to avoid unnecessary computation
    stats_by_ids = {}
    rows = []
    for date in dates:
        stats = {}
        ids = tuple(activity.loc[activity[date] > 0, 'trip_id'])
        if ids in stats_by_ids:
            # Use stats previously computed
            stats = copy(stats_by_ids[ids])
        elif not ids:
            stats  = {col: np.nan for col in cols}
        else:
            # Compute stats afresh
            f = stats[stats['trip_id'].isin(ids)].copy()
            d['num_trips'] = f.shape[0]
            d['num_routes'] = f['route_id'].nunique()
            d['num_stops'] = stop_times.loc[
              stop_times['trip_id'].isin(ids), 'stop_id'].nunique()
            d['service_distance'] = f['distance'].sum()
            d['service_duration'] = f['duration'].sum()
            d['service_speed'] = d['service_distance']/d['service_duration']

            # Compute peak stats, which is the slowest part
            times = np.unique(f[['start_time', 'end_time']].values)
            counts = [hp.count_active_trips(f, t) for t in times]
            start, end = hp.get_peak_indices(times, counts)
            d['peak_num_trips'] = counts[start]
            d['peak_start_time'] = times[start]
            d['peak_end_time'] = times[end]
            
            # Remember stats
            stats_by_ids[ids] = d
        
        d['date'] = date
        rows.append(d)
        
    # Assemble stats into DataFrame
    if rows:
        f = pd.DataFrame(rows)
        f = f[cols].copy()  # Rearrange columns    
        # Convert seconds back to timestrings
        for col in ['peak_start_time', 'peak_end_time']:
            f[col] = f[col].map(lambda t: hp.timestr_to_seconds(t, inverse=True))
    else:
        f = pd.DataFrame([], columns=cols)
        
    return f

In [127]:
%time ts = feed.compute_trip_stats()
dates = feed.get_dates()
len(dates)

CPU times: user 1.79 s, sys: 0 ns, total: 1.79 s
Wall time: 1.79 s


217

In [135]:
#%time p1 = proto1(feed, ts, dates[:1])
%time p2 = proto2(feed, ts, [])
p2

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 9.08 ms


,date,num_trips,num_routes,num_stops,peak_num_trips,peak_start_time,peak_end_time,service_distance,service_duration,service_speed


In [76]:
feed.compute_feed_stats(ts, dates[0])

,num_trips,num_routes,num_stops,peak_num_trips,peak_start_time,peak_end_time,service_distance,service_duration,service_speed
0,622,20,416,39,08:16:00,08:18:00,13774.027234,472.6,29.145212


In [10]:
feed.describe()

,indicator,value
0,agencies,[Department of Transport and Main Roads - Tran...
1,timezone,Australia/Brisbane
2,start_date,20140526
3,end_date,20141228
4,num_routes,22
5,num_trips,1339
6,num_stops,416
7,num_shapes,54
8,sample_date,20140529
9,num_routes_active_on_sample_date,20


In [11]:
feed.summarize()

,table,column,#values,#nonnull_values,#unique_values,min_value,max_value
0,agency,agency_name,1,1,1,Department of Transport and Main Roads - Trans...,Department of Transport and Main Roads - Trans...
1,agency,agency_url,1,1,1,http://www.sunbus.com.au,http://www.sunbus.com.au
2,agency,agency_timezone,1,1,1,Australia/Brisbane,Australia/Brisbane
3,agency,agency_lang,1,1,1,en,en
4,agency,agency_phone,1,1,1,(07)40576411,(07)40576411
0,calendar,service_id,4,4,4,CNS2014-CNS_MUL-Saturday-00,CNS2014-CNS_MUL-Weekday-00-0000100
1,calendar,monday,4,4,2,0,1
2,calendar,tuesday,4,4,2,0,1
3,calendar,wednesday,4,4,2,0,1
4,calendar,thursday,4,4,2,0,1
